In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important }</style>"))

In [2]:
from utilities import *
from models import *

import joblib
from scipy import signal

In [3]:
ble = read_ble('../Data/ble.csv')

In [4]:
coor = np.array([20, 10])

In [5]:
def reg_impute(data):
    for s in scanners:
        dx = coor[0] - ble[s]['X']
        dy = coor[1] - ble[s]['Y']
        d = np.sqrt(np.square(dy) + np.square(dx))
        reg = joblib.load('../Models/linear_distance_models/{}.joblib'.format(s))
        ss = pd.Series(reg.predict(d.reshape((-1, 1)))).repeat(data.shape[0]).reset_index(drop=True)
        data[s].fillna(ss, inplace=True)

In [6]:
def lfilter(xn):
    if isinstance(xn, pd.Series):
        xn = xn.values
    b, a = signal.butter(5, 0.1)
    zi = signal.lfilter_zi(b, a)
    z, _ = signal.lfilter(b, a, xn, zi=zi*xn[0])
    return z

In [7]:
def fn(sub_df):
    for s in scanners:
        sub_df[s] = lfilter(sub_df[s])
    return sub_df

In [8]:
model = MLPRegressor(model_type='special')
model.model.fit(np.array([[0] * 17]), np.array([[0, 0]]))
model.model.load_weights('../Models/MLP_Regression_Rolling_FFill_MinMax_ExtraTest_LinearDistanceModel_FourierTransform.h5')

Train on 1 samples
1/1 [==============================] - 1s 726ms/sample - loss: 0.1325 - mae: 0.0000e+00 - mse: 0.0000e+00


In [9]:
client = InfluxDBClient()

In [10]:
data = client.retrieveData(seconds=5, beacon="0117C55D14E4")

In [11]:
data = format_data(data)

In [12]:
data[scanners] = min_max_scaling(data[scanners])
data.reset_index(drop=True, inplace=True)

In [13]:
reg_impute(data)

In [15]:
fn(data)

,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
0,0.362795,0.490012,0.454016,0.351377,0.475005,0.487223,0.465476,0.285105,0.345214,0.216828,0.366790,0.471217,0.496447,0.192447,0.45032,0.233333,0.328372
1,0.362795,0.490012,0.454016,0.351377,0.475005,0.487223,0.465476,0.285105,0.345214,0.216828,0.366790,0.471217,0.496447,0.192447,0.45032,0.233335,0.328372
2,0.362795,0.490012,0.454016,0.351377,0.475005,0.487223,0.465476,0.285105,0.345214,0.216828,0.366790,0.471223,0.496445,0.192447,0.45032,0.233346,0.328372
3,0.362795,0.490012,0.454016,0.351377,0.475005,0.487223,0.465476,0.285105,0.345214,0.216828,0.366790,0.471245,0.496433,0.192447,0.45032,0.233394,0.328372
4,0.362795,0.490012,0.454016,0.351377,0.475005,0.487229,0.465476,0.285105,0.345214,0.216828,0.366790,0.471299,0.496397,0.192447,0.45032,0.233534,0.328372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,0.346054,0.490127,0.436382,0.352304,0.471831,0.497610,0.475557,0.282951,0.347351,0.211258,0.387108,0.466615,0.495579,0.192447,0.45032,0.260668,0.327608
61,0.347819,0.490191,0.437143,0.351855,0.471970,0.496745,0.478584,0.283355,0.347325,0.215104,0.386753,0.466071,0.495754,0.192447,0.45032,0.258929,0.327087
62,0.350169,0.490228,0.438689,0.351295,0.472358,0.496054,0.481281,0.283802,0.347593,0.218216,0.385921,0.465666,0.495978,0.192447,0.45032,0.257165,0.326776
63,0.352894,0.490237,0.440866,0.350666,0.472916,0.495573,0.483333,0.284254,0.348179,0.220377,0.384627,0.465463,0.496289,0.192447,0.45032,0.255503,0.326666


In [17]:
coor = model.predict(data[scanners].values).mean(axis=0)

In [18]:
np.expand_dims(coor, axis=0)

array([[20.153555,  9.865445]], dtype=float32)

In [19]:
coor

array([20.153555,  9.865445], dtype=float32)

In [20]:
data

,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
0,0.362795,0.490012,0.454016,0.351377,0.475005,0.487223,0.465476,0.285105,0.345214,0.216828,0.366790,0.471217,0.496447,0.192447,0.45032,0.233333,0.328372
1,0.362795,0.490012,0.454016,0.351377,0.475005,0.487223,0.465476,0.285105,0.345214,0.216828,0.366790,0.471217,0.496447,0.192447,0.45032,0.233335,0.328372
2,0.362795,0.490012,0.454016,0.351377,0.475005,0.487223,0.465476,0.285105,0.345214,0.216828,0.366790,0.471223,0.496445,0.192447,0.45032,0.233346,0.328372
3,0.362795,0.490012,0.454016,0.351377,0.475005,0.487223,0.465476,0.285105,0.345214,0.216828,0.366790,0.471245,0.496433,0.192447,0.45032,0.233394,0.328372
4,0.362795,0.490012,0.454016,0.351377,0.475005,0.487229,0.465476,0.285105,0.345214,0.216828,0.366790,0.471299,0.496397,0.192447,0.45032,0.233534,0.328372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,0.346054,0.490127,0.436382,0.352304,0.471831,0.497610,0.475557,0.282951,0.347351,0.211258,0.387108,0.466615,0.495579,0.192447,0.45032,0.260668,0.327608
61,0.347819,0.490191,0.437143,0.351855,0.471970,0.496745,0.478584,0.283355,0.347325,0.215104,0.386753,0.466071,0.495754,0.192447,0.45032,0.258929,0.327087
62,0.350169,0.490228,0.438689,0.351295,0.472358,0.496054,0.481281,0.283802,0.347593,0.218216,0.385921,0.465666,0.495978,0.192447,0.45032,0.257165,0.326776
63,0.352894,0.490237,0.440866,0.350666,0.472916,0.495573,0.483333,0.284254,0.348179,0.220377,0.384627,0.465463,0.496289,0.192447,0.45032,0.255503,0.326666


In [21]:
coor = model.predict(data[scanners]).mean(axis=0)

In [22]:
coor

array([2.2853856, 3.2846942], dtype=float32)